In [ ]:
import pandas as pd
import hdbscan
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import RobustScaler
import optuna
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score, confusion_matrix
import joblib

In [2]:
df = pd.read_csv('../dataset.csv')

**Preprocessing**

*Analize the distribution of the continous columns*

In [ ]:

continuous_cols = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']

# Create subplots for distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Distribution of Continuous Features', fontsize=16)

for idx, col in enumerate(continuous_cols):
    ax = axes[idx//2, idx%2]
    
    # Plot histogram and kernel density estimate
    sns.histplot(data=df, x=col, kde=True, ax=ax)
    
    # Add basic statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    ax.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='--', label=f'Median: {median_val:.2f}')
    ax.legend()
    
plt.tight_layout()

print("\nDistribution Statistics:")
for col in continuous_cols:
    print(f"\n{col}:")
    print(f"Skewness: {stats.skew(df[col]):.3f}")
    print(f"Kurtosis: {stats.kurtosis(df[col]):.3f}")



*Change the Scale of capital-gain and capital-loss and apply robust scaling to all the features*

Considering the distribution of the continuous features (in particular capital-gain and capital-loss), log(1+x) transformation is required. This is done for reducing the extreme values while preserving the relative differences. Considering that HDBSCAN works with distance-based density estimation, extreme values can dominate distance calculations. 

After that, apply RobustScaler so that the features are brought to comparable scales and distances calculations become more meaningful.


In [4]:
df_transformed = df.copy()

# Log1p transformation for financial features
df_transformed['capital-gain'] = np.log1p(df_transformed['capital-gain'])
df_transformed['capital-loss'] = np.log1p(df_transformed['capital-loss'])

# Apply RobustScaler to all numerical features
numerical_features = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = RobustScaler()
df_transformed[numerical_features] = scaler.fit_transform(df_transformed[numerical_features])

#joblib.dump(scaler, "robust_scaler.pkl")


## Uncomment this to plot the distributions after the transformations
# fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# fig.suptitle('Distribution Before and After Transformation', fontsize=16)

# for idx, col in enumerate(['capital-gain', 'capital-loss']):
#     # Original distribution
#     sns.histplot(data=df, x=col, kde=True, ax=axes[idx, 0])
#     axes[idx, 0].set_title(f'Original {col}')
    
#     # Transformed distribution
#     sns.histplot(data=df_transformed, x=col, kde=True, ax=axes[idx, 1])
#     axes[idx, 1].set_title(f'Transformed {col}')

# plt.tight_layout()

In [5]:
#Save the labels and drop them for clustering
X = df_transformed.drop('income', axis=1)  # Features (all columns except 'income')
true_labels = df['income'].values 

**Finding the best set of hyperparameters for HDBSCAN**

Need to prioritize cluster purity, in particular Pattern Discovery (Silhouette score). Using Optuna for finding the best hyperparameters

In [ ]:
def objective(trial, X, true_labels):
    """
    Optuna objective function for optimizing HDBSCAN parameters
    """
    # Define the parameter search space
    min_cluster_size = trial.suggest_int('min_cluster_size', 50, 500)
    min_samples = trial.suggest_int('min_samples', max(5, min_cluster_size//10), min_cluster_size)
    cluster_selection_epsilon = trial.suggest_float('cluster_selection_epsilon', 0.0, 1.0)
    
    # Initialize HDBSCAN with trial parameters
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        cluster_selection_epsilon=cluster_selection_epsilon,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True
    )
    
    # Fit and predict
    cluster_labels = clusterer.fit_predict(X)
    
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    
    # If only one cluster or all points are noise, return worst possible score
    if n_clusters <= 1:
        return -float('inf')
    
    # Calculate silhouette score for non-noise points
    non_noise_mask = cluster_labels != -1
    if non_noise_mask.sum() > 1:
        sil_score = silhouette_score(X[non_noise_mask], cluster_labels[non_noise_mask])
    else:
        sil_score = -float('inf')
    
    # Calculate other metrics
    ari_score = adjusted_rand_score(true_labels, cluster_labels)
    nmi_score = normalized_mutual_info_score(true_labels, cluster_labels)
    noise_ratio = (cluster_labels == -1).sum() / len(cluster_labels)
    
    # Objective score (weights)
    objective_score = (
        0.4 * sil_score +      # Weight for cluster quality (Silhouette score)
        0.3 * ari_score +      # Weight for agreement with true labels
        0.2 * nmi_score -      # Weight for information shared with true labels
        0.1 * noise_ratio      # Small penalty for excessive noise
    )
    
    # Store additional metrics for analysis
    trial.set_user_attr('n_clusters', n_clusters)
    trial.set_user_attr('silhouette', sil_score)
    trial.set_user_attr('ari', ari_score)
    trial.set_user_attr('nmi', nmi_score)
    trial.set_user_attr('noise_ratio', noise_ratio)
    
    return objective_score

def optimize_hdbscan(X, true_labels, n_trials=100):
    """
    Run Optuna optimization for HDBSCAN parameters
    """
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X, true_labels), n_trials=n_trials)
    
    # Get best parameters
    best_params = study.best_trial.params
    
    # Print optimization results
    print("\nOptimization Results:")
    print("Best parameters:", best_params)
    print("Best trial metrics:")
    print(f"Number of clusters: {study.best_trial.user_attrs['n_clusters']}")
    print(f"Silhouette score: {study.best_trial.user_attrs['silhouette']:.3f}")
    print(f"Adjusted Rand Index: {study.best_trial.user_attrs['ari']:.3f}")
    print(f"NMI score: {study.best_trial.user_attrs['nmi']:.3f}")
    print(f"Noise ratio: {study.best_trial.user_attrs['noise_ratio']:.3f}")
    
    return best_params, study

# Usage:
best_params, study = optimize_hdbscan(X, true_labels, n_trials=100)

**Model training**

Once found the best set of hyperparameters, create the model using them. Then evaluate the result obtained by the model

In [6]:
# Initialize HDBSCAN with optimized parameters
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=best_params['min_cluster_size'],
    min_samples=best_params['min_samples'],
    cluster_selection_epsilon=best_params['cluster_selection_epsilon'],
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

# Fit and predict
cluster_labels = clusterer.fit_predict(X)

**Basic metrics**

In [ ]:
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

non_noise_mask = cluster_labels != -1
sil_score = silhouette_score(X[non_noise_mask], cluster_labels[non_noise_mask])


ari_score = adjusted_rand_score(true_labels, cluster_labels)
nmi_score = normalized_mutual_info_score(true_labels, cluster_labels)
noise_ratio = (cluster_labels == -1).sum() / len(cluster_labels)

print(f"Number of clusters: {n_clusters}")
print(f"Silhouette score: {sil_score}")
print(f"Adjusted Rand Index: {ari_score}")
print(f"NMI score: {nmi_score}")
print(f"Noise ratio: {noise_ratio}")

**Analyze intra-cluster distributions and characteristics**

**Analyze the distribution of the income variable**

In [ ]:
def analyze_income_distribution(df, cluster_labels, income_labels):
    df_analysis = pd.DataFrame({
        'Cluster': cluster_labels,
        'Income': income_labels
    })
    
    # Overall cluster composition
    print("Cluster Income Distribution:")
    for cluster in sorted(set(cluster_labels)):
        cluster_mask = df_analysis['Cluster'] == cluster
        cluster_size = cluster_mask.sum()
        
        # Get income distribution for this cluster
        income_dist = df_analysis[cluster_mask]['Income'].value_counts(normalize=True)
        income_counts = df_analysis[cluster_mask]['Income'].value_counts()
        
        cluster_name = "Noise" if cluster == -1 else f"Cluster {cluster}"
        print(f"\n{cluster_name} (Size: {cluster_size})")
        print(f"<=50K: {income_counts.get(0, 0)} ({income_dist.get(0, 0)*100:.1f}%)")
        print(f">50K: {income_counts.get(1, 0)} ({income_dist.get(1, 0)*100:.1f}%)")
        
        if cluster != -1:  # Skip noise points
            correlation = np.corrcoef(cluster_mask, income_labels)[0,1]
            print(f"Correlation with income: {correlation:.3f}")

def plot_income_distribution(cluster_labels, income_labels):
    """
    Visualize income distribution across clusters
    """
    # Create stacked bar chart
    df_plot = pd.DataFrame({
        'Cluster': cluster_labels,
        'Income': ['<=50K' if x == 0 else '>50K' for x in income_labels]
    })
    
    plt.figure(figsize=(10, 6))
    cluster_income = pd.crosstab(df_plot['Cluster'], df_plot['Income'], normalize='index')
    cluster_income.plot(kind='bar', stacked=True)
    plt.title('Income Distribution by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Proportion')
    plt.legend(title='Income')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

def analyze_income_patterns(df, cluster_labels, income_labels):
    
    # Basic distribution analysis
    analyze_income_distribution(df, cluster_labels, income_labels)
    
    # Visualization
    plot_income_distribution(cluster_labels, income_labels)
    
    # Performance metrics
analyze_income_patterns(df_transformed, cluster_labels, true_labels)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap

def visualize_clusters_umap(df_transformed, cluster_labels, sample_size=48843, random_state=31): #sample size equals to actual dataset size (all datapoints plotted)
    """
    Create UMAP visualization with transparent colors to show overlapping regions
    """
    # Sample the data
    if len(df_transformed) > sample_size:
        idx = np.random.RandomState(random_state).choice(
            len(df_transformed), sample_size, replace=False
        )
        df_sample = df_transformed.iloc[idx]
        clusters_sample = cluster_labels[idx]
    else:
        df_sample = df_transformed
        clusters_sample = cluster_labels

    cluster_colors = {
        -1: (1, 0, 1, 0.5),     
        0: (0, 0, 1, 0.5),      
        1: (1, 0, 0, 0.5),     
        2: (0, 0.8, 0, 0.5) 
    }

    # Create UMAP projection
    print("Computing UMAP projection...")
    reducer = umap.UMAP(
        n_neighbors=15,
        min_dist=0.1,
        random_state=random_state
    )
    embedding_umap = reducer.fit_transform(df_sample)

    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Plot clusters with smaller point size and transparency
    for cluster in cluster_labels:
        mask = clusters_sample == cluster
        label = 'Noise' if cluster == -1 else f'Cluster {cluster}'
        plt.scatter(embedding_umap[mask, 0], embedding_umap[mask, 1],
                   color=cluster_colors[cluster],
                   s=20,  # Smaller point size
                   label=label)

    plt.xlabel('UMAP Dimension 1', fontsize=16)
    plt.ylabel('UMAP Dimension 2', fontsize=16)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # Enhanced legend
    plt.legend(fontsize=15,              # Larger font size
              bbox_to_anchor=(0.84, 1),  # Position legend to the right
              loc='upper left',          # Align to upper left
              borderaxespad=0,           # No padding
              frameon=True,              # Add frame
              edgecolor='black',         # Black edge color
              fancybox=True,             # Rounded corners
              shadow=True,
              markerscale=4
              )               # Add shadow

    # Add grid for better readability
    plt.grid(True, alpha=0.3)
    plt.tight_layout()  # Adjust layout to make room for legend
    plt.show()

    # Print cluster sizes
    unique, counts = np.unique(clusters_sample, return_counts=True)
    print("\nCluster Sizes in Sample:")
    for cluster, count in zip(unique, counts):
        label = 'Noise' if cluster == -1 else f'Cluster {cluster}'
        print(f"{label}: {count} points ({count/len(clusters_sample):.1%})")

# Usage:
visualize_clusters_umap(df_transformed, cluster_labels)

**How, by changing a cost function, the demographics get affected?**


 First let's analyze how the demographics (in this case the gender) are distributed in the clusters

In [ ]:
def analyze_demographic_impact(df, cluster_labels, demographic_column):
    """
    Analyze how clustering affects different demographic groups
    """
    impact_analysis = pd.DataFrame({
        'Cluster': cluster_labels,
        'Demographic': df[demographic_column]
    })
    
    # Distribution of demographics across clusters
    cluster_demographics = pd.crosstab(
        impact_analysis['Cluster'], 
        impact_analysis['Demographic'],
        normalize='index'
    )
    
    # Noise point analysis by demographic
    noise_rates = impact_analysis[impact_analysis['Cluster'] == -1]['Demographic'].value_counts(normalize=True)
    
    return cluster_demographics, noise_rates

analyze_demographic_impact(df, cluster_labels, 'gender')

Run the new model with the parameters changed

In [14]:
# Initialize HDBSCAN with different cluster size (440 instead of 220)
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=440,
    min_samples=117,
    cluster_selection_epsilon=0.28479667859306007,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

# Fit and predict
cluster_labels = clusterer.fit_predict(X)

In [ ]:
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

non_noise_mask = cluster_labels != -1
sil_score = silhouette_score(X[non_noise_mask], cluster_labels[non_noise_mask])


ari_score = adjusted_rand_score(true_labels, cluster_labels)
nmi_score = normalized_mutual_info_score(true_labels, cluster_labels)
noise_ratio = (cluster_labels == -1).sum() / len(cluster_labels)

print(f"Number of clusters: {n_clusters}")
print(f"Silhouette score: {sil_score}")
print(f"Adjusted Rand Index: {ari_score}")
print(f"NMI score: {nmi_score}")
print(f"Noise ratio: {noise_ratio}")

In [ ]:
unique_clusters, counts = np.unique(cluster_labels, return_counts=True)
for cluster, count in zip(unique_clusters, counts):
    cluster_name = 'Noise' if cluster == -1 else f'Cluster {cluster}'
    print(f"{cluster_name}: {count} points ({count/len(cluster_labels):.2%})")

In [ ]:
analyze_demographic_impact(df, cluster_labels, 'gender')

**Export the model**

In [ ]:
import joblib

joblib.dump(clusterer, 'hdbscan_model.joblib')


In [31]:
import pickle

filename = 'hdbscan_model.pkl'
pickle.dump(clusterer, open(filename, 'wb'))